# **SQL Query using Pyspark**

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
# !wget https://github.com/tmclouisluk/FIFA-players-Data-Analysis/blob/master/pre_model_df.csv

--2023-08-22 03:59:28--  https://github.com/tmclouisluk/FIFA-players-Data-Analysis/blob/master/pre_model_df.csv
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5403 (5.3K) [text/plain]
Saving to: ‘pre_model_df.csv’

pre_model_df.csv    100%[===================>]   5.28K  --.-KB/s    in 0s      

2023-08-22 03:59:28 (65.9 MB/s) - ‘pre_model_df.csv’ saved [5403/5403]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SQLonPyspark").getOrCreate()
data = spark.read.csv("/content/sample_data/california_housing_train.csv", header=True, inferSchema=True)
data.show()

# **SELECT QEURY**

In [ ]:
data.createOrReplaceTempView("housingdata")
result = spark.sql("SELECT total_rooms, population FROM housingdata")
result.show(5)

+-----------+----------+
|total_rooms|population|
+-----------+----------+
|     5612.0|    1015.0|
|     7650.0|    1129.0|
|      720.0|     333.0|
|     1501.0|     515.0|
|     1454.0|     624.0|
+-----------+----------+
only showing top 5 rows



# **Filter with WHERE clause**

In [ ]:
result_filtered = spark.sql("SELECT longitude, latitude, population FROM housingdata WHERE population < 1000")
result_filtered.show(5)

+---------+--------+----------+
|longitude|latitude|population|
+---------+--------+----------+
|  -114.56|   33.69|     333.0|
|  -114.57|   33.64|     515.0|
|  -114.57|   33.57|     624.0|
|  -114.58|   33.63|     671.0|
|  -114.59|   34.83|     375.0|
+---------+--------+----------+
only showing top 5 rows



# **Aggregation Function**

In [ ]:
result_aggregated = spark.sql("SELECT AVG(population) AS avg_population FROM housingdata")
result_aggregated.show()


+------------------+
|    avg_population|
+------------------+
|1429.5739411764705|
+------------------+



# **ORDER BY**

In [ ]:
result_ordered = spark.sql("SELECT longitude, latitude, total_rooms FROM housingdata ORDER BY total_rooms DESC")
result_ordered.show(5)

+---------+--------+-----------+
|longitude|latitude|total_rooms|
+---------+--------+-----------+
|  -117.74|   33.89|    37937.0|
|  -121.79|   36.64|    32627.0|
|  -117.78|   34.03|    32054.0|
|  -118.78|   34.16|    30405.0|
|  -117.12|   33.52|    30401.0|
+---------+--------+-----------+
only showing top 5 rows



# **JOINS**

In [ ]:
data2 = [("Alice", "Engineering"), ("Bob", "Marketing"), ("David", "Finance")]
columns2 = ["name", "department"]
df2 = spark.createDataFrame(data2, columns2)

df2.createOrReplaceTempView("departments")

data = [("Alice", 25), ("Bob", 30), ("Charlie", 22)]
columns = ["name", "age"]
df = spark.createDataFrame(data, columns)

df.createOrReplaceTempView("people")

join_query = "SELECT p.name, p.age, d.department FROM people p JOIN departments d ON p.name = d.name"
result_join = spark.sql(join_query)
result_join.show()

+-----+---+-----------+
| name|age| department|
+-----+---+-----------+
|  Bob| 30|  Marketing|
|Alice| 25|Engineering|
+-----+---+-----------+



# **Sub Queries**

In [ ]:
subquery = "SELECT total_rooms, total_bedrooms, population FROM housingdata WHERE population > (SELECT AVG(population) FROM housingdata)"
result_subquery = spark.sql(subquery)
result_subquery.show(5)

+-----------+--------------+----------+
|total_rooms|total_bedrooms|population|
+-----------+--------------+----------+
|     2907.0|         680.0|    1841.0|
|     4789.0|        1175.0|    3134.0|
|     3741.0|         801.0|    2434.0|
|     1706.0|         397.0|    3424.0|
|     3414.0|         666.0|    2097.0|
+-----------+--------------+----------+
only showing top 5 rows



# **UPDATE Queries**

In [ ]:
# new_age = 40
# update_condition = "name = 'Bob'"

# update_query = f"""
#     INSERT OVERWRITE TABLE people
#     SELECT
#         name,
#         CASE WHEN {update_condition} THEN {new_age} ELSE age END AS age
#     FROM people
# """

# # update_query = "UPDATE people SET age = 40 WHERE name = 'Bob'"
# spark.sql(update_query)

# updated_df = spark.sql("SELECT * FROM people")
# updated_df.show(5)

AnalysisException: ignored

# **Insert New Attributes**

In [ ]:
data = [("Alice", 25), ("Bob", 30), ("Charlie", 22)]
columns = ["name", "age"]
df = spark.createDataFrame(data, columns)

print("Original DataFrame:")
df.show()

df.createOrReplaceTempView("people")

new_attributes_query = """
    SELECT
        name,
        age,
        'Engineering' AS department  -- New department column with constant value
    FROM people
"""

# Execute the query to create a new DataFrame with the new column
new_attributes_df = spark.sql(new_attributes_query)

# Display the new DataFrame
print("DataFrame with New Attributes:")
new_attributes_df.show()


Original DataFrame:
+-------+---+
|   name|age|
+-------+---+
|  Alice| 25|
|    Bob| 30|
|Charlie| 22|
+-------+---+

DataFrame with New Attributes:
+-------+---+-----------+
|   name|age| department|
+-------+---+-----------+
|  Alice| 25|Engineering|
|    Bob| 30|Engineering|
|Charlie| 22|Engineering|
+-------+---+-----------+



# **Date and Time**

In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

data=[["1","2020-02-01"],["2","2019-03-01"],["3","2021-03-01"]]
df=spark.createDataFrame(data,["id","input"])
df.show()

+---+----------+
| id|     input|
+---+----------+
|  1|2020-02-01|
|  2|2019-03-01|
|  3|2021-03-01|
+---+----------+



# **Current Date Show**

In [ ]:
df.select(current_date().alias("current_date")
  ).show(1)

+------------+
|current_date|
+------------+
|  2023-08-22|
+------------+
only showing top 1 row



# **Date Formating**

In [ ]:
df.select(col("input"),
    date_format(col("input"), "MM-dd-yyyy").alias("date_format")
  ).show()

+----------+-----------+
|     input|date_format|
+----------+-----------+
|2020-02-01| 02-01-2020|
|2019-03-01| 03-01-2019|
|2021-03-01| 03-01-2021|
+----------+-----------+



# **Date Difference**

In [ ]:

df.select(col("input"),
    datediff(current_date(),col("input")).alias("datediff")
  ).show()


+----------+--------+
|     input|datediff|
+----------+--------+
|2020-02-01|    1298|
|2019-03-01|    1635|
|2021-03-01|     904|
+----------+--------+



# **Day, Month, Year**

In [ ]:
df.select(col("input"),
     year(col("input")).alias("year"),
     month(col("input")).alias("month"),
     weekofyear(col("input")).alias("weekofyear")
  ).show()

+----------+----+-----+----------+
|     input|year|month|weekofyear|
+----------+----+-----+----------+
|2020-02-01|2020|    2|         5|
|2019-03-01|2019|    3|         9|
|2021-03-01|2021|    3|         9|
+----------+----+-----+----------+



# **Timestamp**

In [ ]:
data=[["1","2020-02-01 11:01:19.06"],["2","2019-03-01 12:01:19.406"],["3","2021-03-01 12:01:19.406"]]
df3=spark.createDataFrame(data,["id","input"])

df3.select(col("input"),
    hour(col("input")).alias("hour"),
    minute(col("input")).alias("minute"),
    second(col("input")).alias("second")
  ).show(truncate=False)

+-----------------------+----+------+------+
|input                  |hour|minute|second|
+-----------------------+----+------+------+
|2020-02-01 11:01:19.06 |11  |1     |19    |
|2019-03-01 12:01:19.406|12  |1     |19    |
|2021-03-01 12:01:19.406|12  |1     |19    |
+-----------------------+----+------+------+

